In [1]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import os

In [2]:
"""
Implementation of the noiseless VQE algorithm to find the ground state of water (H₂O) using both single and double excitations.
"""

'\nImplementation of the noiseless VQE algorithm to find the ground state of water (H₂O) using both single and double excitations.\n'

In [3]:
# Define the atoms in the water molecule
symbols = ["O", "H", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.000000, 0.000000, 0.000000],  # Oxygen at origin
    [0.758602, 0.000000, 0.504284],  # Hydrogen 1
    [-0.758602, 0.000000, 0.504284]   # Hydrogen 2
])

In [4]:
# Define the water Hamiltonian and the number of qubits required
# Default STO-3G basis set
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0)

# 1 spatial orbital for each H and 5 for O
# Each spatial orbital is doubly-degenerate (2 spin orbitals)
print(f"{qubits} qubits required")

14 qubits required


In [ ]:
# Which orbitals are occupied (1) or unoccupied (0) in the mean-field (Hartree-Fock) solution
electrons = 10  # 1 from each H and 8 from O
hf = qchem.hf_state(electrons=electrons, orbitals=qubits)

# Show that the first 10 orbitals are occupied
# Print the state in bra-ket notation
print(f"Hartree-Fock state: {'|' + ''.join(str(bit) for bit in hf) + '⟩'}")

In [ ]:
# Define the number of required quantum wires / logical qubits
num_wires = qubits

# Create quantum device simulator backend
dev = qml.device("default.qubit",  # Noiseless state vector simulator
                 wires=num_wires)


@qml.qnode(dev)  # Transforms exp_energy below into a quantum node
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))

    # Return the expectation value of the molecular Hamiltonian
    return qml.expval(hamiltonian)

# Calculate ⟨ψ_hf| hamiltonian |ψ_hf⟩ in Hartree (Ha) units
# 1 Ha = 27.2 eV
exp_energy(hf)

In [ ]:
# Generate excitation indices
singles, doubles = excitations(electrons, qubits)

# Define an ansatz using the excitations
def ansatz(params_singles, params_doubles):
    # Initialize the qubit register in the Hartree-Fock state
    qml.BasisState(hf, wires=range(num_wires))
    
    # Apply single excitations
    for i, excitation in enumerate(singles):
        qml.SingleExcitation(params_singles[i], wires=excitation)
    
    # Apply double excitations
    for i, excitation in enumerate(doubles):
        qml.DoubleExcitation(params_doubles[i], wires=excitation)

In [ ]:
# Define the VQE cost function
@qml.qnode(dev)
def cost_function(params_singles, params_doubles):
    ansatz(params_singles, params_doubles)

    # Measure the expectation value of the Hamiltonian after applying the ansatz:
    # E(theta) = ⟨ψ(theta)| H |ψ(theta)⟩
    return qml.expval(hamiltonian)

# Initialize both parameter sets to zero-vectors
initial_params_singles = np.zeros(len(singles), requires_grad=True)
initial_params_doubles = np.zeros(len(doubles), requires_grad=True)

# Confirm the initial energy of the system using the Hartree-Fock state
# This is the starting point for classical optimization
cost_function(initial_params_singles, initial_params_doubles)

In [ ]:
# Adaptive Moment Estimation (Adam) optimizer
opt = qml.AdamOptimizer(stepsize=0.1)

# Initialize zero-vectors as the starting parameters for the ansatz
theta_singles = np.zeros(len(singles), requires_grad=True)
theta_doubles = np.zeros(len(doubles), requires_grad=True)

# Initialize energy and parameter tracking lists
energy = [cost_function(theta_singles, theta_doubles)]
angle_singles = [theta_singles]
angle_doubles = [theta_doubles]

# VQE optimization loop
max_iterations = 50
for n in range(max_iterations):
    # Updates parameters theta to minimize the cost function (energy)
    (theta_singles, theta_doubles), prev_energy = opt.step_and_cost(
        cost_function, theta_singles, theta_doubles
    )
    energy.append(cost_function(theta_singles, theta_doubles))
    angle_singles.append(theta_singles)
    angle_doubles.append(theta_doubles)

# Plotting VQE convergence
plt.plot(range(len(energy)), energy)
plt.xlabel('Iteration')
plt.ylabel('Energy (Ha)')
plt.title('VQE Convergence')

# Ensure the "images" directory exists
os.makedirs('images', exist_ok=True)

# Save the figure to the "images" directory
plt.savefig('images/H2O_Adam.png')
plt.show()

In [ ]:
print(f"Final ground state energy: {energy[-1]:.8f} Ha")
print(f"Final single excitation parameters: {theta_singles[0]:.8f}, {theta_singles[1]:.8f}")
print(f"Final double excitation parameters: {theta_doubles[0]:.8f}, {theta_doubles[1]:.8f}")

In [ ]:
@qml.qnode(dev)
def ground_state(params_singles, params_doubles):
    ansatz(params_singles, params_doubles)

    # Return the entire quantum statevector
    return qml.state()

final_state = ground_state(theta_singles, theta_doubles)

# Find non-zero (or near non-zero) entries
threshold = 1e-2  # Recommended smaller threshold to capture all significant amplitudes
non_zero_indices = np.where(np.abs(final_state) > threshold)[0]
non_zero_amplitudes = final_state[non_zero_indices]

# Build the full ket notation string
ket_terms = []
for idx, amp in zip(non_zero_indices, non_zero_amplitudes):
    binary_state = f"|{idx:0{num_wires}b}>"
    
    # Format the amplitude nicely
    amp_str = f"{amp.real:.4f}" if np.isclose(amp.imag, 0, atol=1e-6) else f"({amp.real:.4f} + {amp.imag:.4f}j)"
    ket_terms.append(f"{amp_str}{binary_state}")

# Join all terms into one quantum state expression and print
ket_notation = " + ".join(ket_terms)
print(f"Ground state of H₂O:\n|ψ⟩ = {ket_notation}")

In [ ]:
# Prepare labels and amplitudes for the plot
labels = [f"|{idx}⟩" for idx in non_zero_indices]  # No extra brackets

amplitudes = np.abs(non_zero_amplitudes)

# Bar plot
plt.figure(figsize=(12, 6))
plt.bar(labels, amplitudes)
plt.xlabel('Basis States', fontsize=14)
plt.ylabel('Amplitude', fontsize=14)
plt.title('Ground State of H₂O (VQE Result)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Save the figure to the "images" directory
plt.savefig('images/H2O_Ground_State.png')
plt.show()